In [15]:
#Create a dataset
items = ['milk', 'cheese', 'butter', 'yogurt', 'cream', 'bread', 'croissant', 'bagel', 'brioche', 'aspirin', 
         'ibuprofen', 'paracetamol', 'antibiotic', 'vitamin', 'cough', 'syrup', 'tablet', 'ointment', 'rice', 
         'pasta', 'cereal', 'flour', 'sugar', 'oil', 'grain', 'seasoning', 'spice', 'chicken', 'beef', 'pork', 
         'fish', 'salmon', 'tuna', 'shrimp', 'bacon', 'sausage', 'apple', 'banana', 'carrot', 'lettuce', 'tomato', 
         'berry', 'orange', 'peach', 'grape', 'water', 'juice', 'soda', 'coffee', 'tea', 'beer', 'wine', 'liquor', 
         'drink', 'shampoo', 'soap', 'lotion', 'perfume', 'makeup', 'deodorant', 'lipstick', 'nail']

quantity_descriptors = [
    "1-liter of", "500g of", "Pack of 12", "Single", "Pack of 6", "Dozen",
    "30", "100 tablets", "10 sachets",
    "1kg of", "500g packet of", "2lb bag of", "500g of", "1kg packet of", "Bag of 5", "1-liter bottle of",
    "6-pack of", "Case of 24", "100ml of", "250ml bottle of", "Tube of"
]


In [16]:
TRAIN_DATA = [
    (f"{q} {item}", {"entities": [(0, len(q) if q.find(' ') == -1 else q.find(' '), "QUANTITY")]})
    for q, item in zip(quantity_descriptors, items)
]

print(TRAIN_DATA)
print(len(TRAIN_DATA))


[('1-liter of milk', {'entities': [(0, 7, 'QUANTITY')]}), ('500g of cheese', {'entities': [(0, 4, 'QUANTITY')]}), ('Pack of 12 butter', {'entities': [(0, 4, 'QUANTITY')]}), ('Single yogurt', {'entities': [(0, 6, 'QUANTITY')]}), ('Pack of 6 cream', {'entities': [(0, 4, 'QUANTITY')]}), ('Dozen bread', {'entities': [(0, 5, 'QUANTITY')]}), ('30 croissant', {'entities': [(0, 2, 'QUANTITY')]}), ('100 tablets bagel', {'entities': [(0, 3, 'QUANTITY')]}), ('10 sachets brioche', {'entities': [(0, 2, 'QUANTITY')]}), ('1kg of aspirin', {'entities': [(0, 3, 'QUANTITY')]}), ('500g packet of ibuprofen', {'entities': [(0, 4, 'QUANTITY')]}), ('2lb bag of paracetamol', {'entities': [(0, 3, 'QUANTITY')]}), ('500g of antibiotic', {'entities': [(0, 4, 'QUANTITY')]}), ('1kg packet of vitamin', {'entities': [(0, 3, 'QUANTITY')]}), ('Bag of 5 cough', {'entities': [(0, 3, 'QUANTITY')]}), ('1-liter bottle of syrup', {'entities': [(0, 7, 'QUANTITY')]}), ('6-pack of tablet', {'entities': [(0, 6, 'QUANTITY')]}), (

In [17]:
products = [
    {"id": 1, "title": "Pack of 10 - Premium Cotton Towels"},
    {"id": 2, "title": "Set of Three Handcrafted Pottery Vases"},
    {"id": 3, "title": "5-Gallon High-Efficiency Concentrated Laundry Detergent"},
    {"id": 4, "title": "Bundle of 50 LED Light Bulbs - Energy Saving"},
    {"id": 5, "title": "Double Pack: Two 24oz Stainless Steel Water Bottles"},
    {"id": 6, "title": "Apple iPhone 11 64GB Red Unlocked - 3 Network"},
    {"id": 7, "title": "Amazon Kindle 2019 Edition"}
]

# Function to find numbers and associated phrases indicating quantities
def extract_quantity(text):
    import re
    # This regex pattern is quite simplistic and may need to be refined for more complex scenarios
    pattern = r'(\b\d+-?[a-zA-Z]*|\b[Tt]wo|\b[Tt]hree|\b[Ff]our|\b[Ff]ive|\b[Ss]ix|\b[Ss]even|\b[Ee]ight|\b[Nn]ine|\b[Tt]en)\b'
    matches = re.finditer(pattern, text)
    return [(match.start(), match.end(), 'QUANTITY') for match in matches]

# Generate training data
for product in products:
    title = product['title']
    entities = extract_quantity(title)
    if entities:
        TRAIN_DATA.append((title, {"entities": entities}))

print(TRAIN_DATA)
print(len(TRAIN_DATA))


[('1-liter of milk', {'entities': [(0, 7, 'QUANTITY')]}), ('500g of cheese', {'entities': [(0, 4, 'QUANTITY')]}), ('Pack of 12 butter', {'entities': [(0, 4, 'QUANTITY')]}), ('Single yogurt', {'entities': [(0, 6, 'QUANTITY')]}), ('Pack of 6 cream', {'entities': [(0, 4, 'QUANTITY')]}), ('Dozen bread', {'entities': [(0, 5, 'QUANTITY')]}), ('30 croissant', {'entities': [(0, 2, 'QUANTITY')]}), ('100 tablets bagel', {'entities': [(0, 3, 'QUANTITY')]}), ('10 sachets brioche', {'entities': [(0, 2, 'QUANTITY')]}), ('1kg of aspirin', {'entities': [(0, 3, 'QUANTITY')]}), ('500g packet of ibuprofen', {'entities': [(0, 4, 'QUANTITY')]}), ('2lb bag of paracetamol', {'entities': [(0, 3, 'QUANTITY')]}), ('500g of antibiotic', {'entities': [(0, 4, 'QUANTITY')]}), ('1kg packet of vitamin', {'entities': [(0, 3, 'QUANTITY')]}), ('Bag of 5 cough', {'entities': [(0, 3, 'QUANTITY')]}), ('1-liter bottle of syrup', {'entities': [(0, 7, 'QUANTITY')]}), ('6-pack of tablet', {'entities': [(0, 6, 'QUANTITY')]}), (

In [18]:
import spacy

# Load a pre-trained model or create a new one
nlp = spacy.blank("en")  # For a new model
# nlp = spacy.load("en_core_web_sm")  # To update an existing model

# Add the NER pipeline if it's not already there
if "ner" not in nlp.pipe_names:
    nlp.add_pipe('ner')  # Add the NER pipe this way
ner = nlp.get_pipe("ner")

# Add the new entity type to the pipeline
ner.add_label("QUANTITY")


1

In [19]:
from spacy.training import Example
import random

# Prepare the training data
examples = [Example.from_dict(nlp.make_doc(text), annotations) for text, annotations in TRAIN_DATA]

# Train the model
optimizer = nlp.begin_training()
for itn in range(10):  # Number of training iterations
    random.shuffle(examples)
    losses = {}
    for example in examples:
        nlp.update([example], drop=0.5, losses=losses)
    print(losses)


{'ner': 70.54023786378093}
{'ner': 51.443430895451456}
{'ner': 48.272044919780456}
{'ner': 45.21844068257633}
{'ner': 33.7901992043992}
{'ner': 28.776486041644663}
{'ner': 30.37015892824453}
{'ner': 23.058868124638277}
{'ner': 29.056196646997787}
{'ner': 23.55485140323574}


In [20]:
test_text = "Box of 3 Men's T Shirts"
doc = nlp(test_text)
for ent in doc.ents:
    print(ent.text, ent.label_)


Box QUANTITY
3 QUANTITY
T QUANTITY
